# Limpieza de base de datos de Sharks (ataques de tiburón a nivel mundial)

Requisitos:

1) Borrar columnas sí y sólo sí len(NaN)>80%

2) Mínimo de filas: 6000

3) Incluir: eliminar duplicados, tener mismo tipo de dato, eliminar valores nulos...


Importamos las librerías que podemos llegar a usar

In [1]:
%pip install seaborn

%matplotlib inline

%pip install requests 

%pip install openpyxl

%pip install xlrd

%pip install regex

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import regex as re

import pylab as plt
import seaborn as sns

Cargamos la base de datos como csv para trabajarlo

In [3]:
df = pd.read_csv('../Data/attacks.csv', encoding='latin1')

In [4]:
df.head(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


Se procede a definir parámetros básicos del df así como a crear una copia para trabajarla sin tocar la original

In [5]:
df1=df.copy()

In [6]:
df1.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
df1.shape

(25723, 24)

#                                                            LIMPIEZA 

1) Comprobar si hay duplicados y eliminarlos para trabajar sobre datos particulares

In [8]:
df1.drop_duplicates(inplace=True) 

In [9]:
df1.shape

(6312, 24)

La lista contenía muchas filas vacías de forma repetitiva que no aportaban información por lo que se han eliminado

2) Búsqueda de nulos para tomar decisiones de eliminación de datos. Primero por columnas y luego por filas

In [10]:
nancols = df1.isnull().sum()

nancols[nancols>0]

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

Se observa que las columnas 'unnamed: 22' y 'unnamed: 23' tienen casi la totalidad de sus datos nulos por lo que podemos prescindir de estas dos columnas ya que no aportan información. El resto de columnas tienen menos del 80% de nulos por lo que no podemos eliminarlas según los requisitos

In [11]:
df1.drop(['Unnamed: 22','Unnamed: 23'], axis=1, inplace=True)

df1.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

Se procede a comprobar si hay filas con elevado numero de nulos (>90% de 22 columnas) para eliminarlas al no aportar información

In [12]:
nanrows = df1.isnull().sum(axis=1)

nanrows= nanrows[nanrows>=20]

nanrows

6302     20
6303     20
6304     20
6305     20
6306     20
6307     20
6308     20
6309     21
8702     22
25722    21
dtype: int64

In [13]:
nanrowslist= list(nanrows.index)

In [14]:
df1.drop(nanrowslist, axis=0, inplace=True)

In [15]:
df1.shape

(6302, 22)

3) Búsqueda de nulos y reemplazo de valores. Ya se ha comprobado que hay valores nulos en muchas columnas así que hay que reemplazarlos por otros valores

In [16]:
nancols

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

In [17]:
nancols.drop(['Unnamed: 22', 'Unnamed: 23'], inplace=True)

In [18]:
nancols

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
dtype: int64

Los datos nulos de estas columnas se sustuirán por 'unknown' o'not declared' para tener la tabla sin vacíos pero hay que renombrar algunos nombres de columna para poder referirnos a ellas (contienen espacios)

In [19]:
df1.columns=[c.lower().replace(' ', '_') for c in df1.columns]

df1.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex_', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species_', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order'],
      dtype='object')

Se ha encontrado que la variable sex y species tenían un espacio que ha sido renombrado con un '_'. También se modifica la variable fatal_(y/n) por error de transcripción. Se han modificado para tener el nombre correctamente

In [120]:
df1.rename(columns={'sex_': 'sex','species_': 'species', 'fatal_(y/n)': 'fatal'}, inplace=True)

df1.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal', 'time', 'species',
       'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'ageok'],
      dtype='object')

Se cambia el nombre de los nulos de las columnas categóricas y alguna numérica: se añade 'not declared' a los datos nulos sobre datos personales de los afectados y 'unknown' para el resto de valores nulos que son de origen desconocidos

In [21]:
df1.fillna({'case_number': 'unknown', 'type': 'unknown', 'country': 'unknown', 'area': 'unknown', 'location': 'unknown', 'activity': 'unknown', 'name': 'not declared', 'sex_': 'not declared', 'injury': 'unknown', 'fatal_(y/n)': 'unknown', 'time': 'unknown', 'species_': 'unknown', 'investigator_or_source': 'unknown', 'pdf': 'unknown', 'href_formula': 'unknown', 'href': 'unknown'}, inplace=True)

Para columnas numéricas tales como date, year and age se investiga primero el tipo de dato y se trabaja a partir de él. Empezamos con year, que es float y lo vamos a pasar a int para tener todos los valores por año correctamente. Sus valores nulos serán ceros para poder trabajar con la columna.

In [22]:
df1.year.unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [23]:
df1.fillna({'year': 0}, inplace=True)

In [24]:
df1.year.unique()

array([2018., 2017.,    0., 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [25]:
df1['year'] = df1['year'].astype(int)

Para los valores nulos de age se van a sustituir por un valor, pero hay que valorar si es mejor la media, la mediana o la moda. Para ello vamos a establecer un gráfico de distribución de puntos pero antes tenemos que arreglar la columna de age, ya que contiene todo tipo de datos.

In [26]:
df1.age.value_counts()

17                154
18                150
19                142
20                141
15                139
                 ... 
20?                 1
 28                 1
7      &    31      1
 30                 1
13 or 14            1
Name: age, Length: 157, dtype: int64

Se crea una nueva columna con los valores numéricos extraídos de la columna age con regex.

In [57]:
df1["ageok"]= df1["age"].str.findall('[0-9]+')

df1.ageok.value_counts()

[17]        154
[18]        151
[20]        151
[19]        143
[15]        139
           ... 
[87]          1
[67]          1
[21, 26]      1
[18, 22]      1
[13, 14]      1
Name: ageok, Length: 117, dtype: int64

Como puede verse, había registros que incluían más de una persona en el mismo registro, por lo que deberían registrarse nuevas entradas en el dataframe para estas personas

In [77]:
df1['ageok'] = df1['ageok'].apply(lambda [x]: x)

SyntaxError: invalid syntax (4233583578.py, line 1)

In [ ]:
sns.boxplot(df1.age)

In [75]:
df1.ageok.info

<bound method Series.info of 0       [57]
1       [11]
2       [48]
3        NaN
4        NaN
        ... 
6297     NaN
6298     NaN
6299     NaN
6300     NaN
6301    [15]
Name: ageok, Length: 6302, dtype: object>

4) Valores atípicos, incongruentes y/o mal reportados. Hay valores que están mal escritos, detallados o que no pueden tenerse en cuenta al no estar incluídos como deberían. Vamos a arreglarlos.

Por ejemplo, en la columna de 'sex' hay 6 valores que no encajan con las dos posibilidades (M/F). Vamos a asumir que 'M ' y 'N' se refieren a 'M, y descartar 'lli' y '.' como valores

In [87]:
df1.sex.value_counts()

M               5094
F                637
not declared     565
M                  2
N                  2
lli                1
.                  1
Name: sex, dtype: int64

In [91]:
df1.sex.replace({'M ': 'M', 'N': 'M', 'lli': 'not declared', '.': 'not declared'}, inplace=True)

df1.sex.value_counts()

M               5098
F                637
not declared     567
Name: sex, dtype: int64

Repetimos este proceso pero para los valores de Y/N de fatal. Se unifican los 'unknown' en minúscula, se devuelven los ' N', 'M' y 'N ' como 'N', y el 'y' como 'Y'. El dato numérico '2017' se cambia a 'unknown'

In [ ]:
df1.fatal.value_counts()

In [125]:
df1.fatal.replace({' N': 'N', 'N ': 'N', 'M ': 'N', 'y': 'Y', 'UNKNOWN': 'unknown', '2017': 'unknown', 'M': 'N'}, inplace=True)

df1.fatal.value_counts()

N          4302
Y          1389
unknown     611
Name: fatal, dtype: int64

In [126]:
df1

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,ageok
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,[57]
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,[11]
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,[48]
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,unknown,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,unknown,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,unknown,unknown,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN
6298,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,unknown,Pearl diving,Ahmun,M,NaN,FATAL,Y,unknown,unknown,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN
6299,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,unknown,unknown,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN
6300,ND.0002,1883-1889,0,Unprovoked,PANAMA,unknown,"Panama Bay 8ºN, 79ºW",unknown,Jules Patterson,M,NaN,FATAL,Y,unknown,unknown,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN


5) Establecer nuevas variables. En base a la fecha aportada por la columna date, se extraen las columnas de día, mes y año (que ya está registrada, por lo que habrá que comprobar si es correcta). De esta manera se podrán estudiar estas variables independientemente, siempre que estén registradas como tal en la columna date.